In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-10-02 18:56:33--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.48MB/s    in 0.2s    

2021-10-02 18:56:34 (4.48 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
softwares_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Software_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
softwares_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
HPC_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
HPC_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [7]:
#Amazon Softwares Vine Reviews
from pyspark import SparkFiles
vine_table_softwares = softwares_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_softwares.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3EFW2STIYIY0I|          1|            2|          2|   N|
|R12NR0R5A9F7FT|          5|            0|          0|   N|
|R1LSH74R9XAP59|          2|            0|          1|   N|
|R1QXUNTF76K7L6|          2|            0|          0|   N|
|R2F7DR75PS8NKT|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [8]:
#Amazon Health Personal Care's Vine Reviews
from pyspark import SparkFiles
vine_table_HPC = HPC_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_HPC.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R3EQSTM9PWRAL|          3|            0|          0|   N|
| RBWPRK17XKIXD|          5|            0|          0|   N|
| RRSLOAF273XFC|          5|            1|          2|   N|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



Droping NA & Duplicates

In [9]:
print(vine_table_softwares.count())
vine_table_softwares = vine_table_softwares.dropna()
vine_table_softwares = vine_table_softwares.na.drop()
vine_table_softwares = vine_table_softwares.dropDuplicates()
print(vine_table_softwares.count())

341931
341931


In [10]:
print(vine_table_HPC.count())
vine_table_HPC = vine_table_HPC.dropna()
vine_table_HPC = vine_table_HPC.na.drop()
vine_table_HPC = vine_table_HPC.dropDuplicates()
print(vine_table_HPC.count())

5331449
5331215


In [11]:
#Softwares
vine_table_softwares.describe().show()

+-------+--------------+------------------+------------------+------------------+------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes|  vine|
+-------+--------------+------------------+------------------+------------------+------+
|  count|        341931|            341931|            341931|            341931|341931|
|   mean|          null|3.5670354545215264| 4.362482489157169| 5.503203862767634|  null|
| stddev|          null|1.6089364873873202|19.147154630924017|20.681602006116535|  null|
|    min|R1000QJKXGZITK|                 1|                 0|                 0|     N|
|    max| RZZZ6LR5CYS9O|                 5|                99|                99|     Y|
+-------+--------------+------------------+------------------+------------------+------+



In [12]:
# Health Personal Cares
vine_table_HPC.describe().show()

+-------+--------------+------------------+------------------+------------------+-------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes|   vine|
+-------+--------------+------------------+------------------+------------------+-------+
|  count|       5331215|           5331215|           5331215|           5331215|5331215|
|   mean|          null| 4.161819210067499|2.3020221469214803|2.9970260062668643|   null|
| stddev|          null|1.3241704571172515|22.013237763127364|23.645529677600788|   null|
|    min|R10000VJGO8MCK|                 1|                 0|                 0|      N|
|    max| RZZZZJURU8XGH|                 5|               998|               999|      Y|
+-------+--------------+------------------+------------------+------------------+-------+



Average Star Rating for Vine Members vs Non Members



In [17]:
# Softwares
from pyspark.sql.types import IntegerType
from pyspark import SparkFile
vine_table_softwares = vine_table_softwares.withColumn("star_rating",softwares_df["star_rating"].cast(IntegerType()))
vine_table_softwares.groupBy("vine").avg("star_rating").show(truncate=False)

ImportError: ignored

In [ ]:
# Health Personal Care
from pyspark.sql.types import IntegerType
from pyspark import SparkFiles
vine_table_HPC = vine_table_HPC.withColumn("star_rating",HPC_df["star_rating"].cast(IntegerType()))
vine_table_HPC.groupBy("vine").avg("star_rating").show(truncate=False)

Number of five stars reviews



In [ ]:
# Softwares
from pyspark import SparkFiles
from pyspark.sql.functions import avg, col, desc
vine_table_softwares.groupBy("star_rating", "vine").count().sort(col("star_rating").desc()).show(truncate=False)

In [ ]:
# Health Personal Care
from pyspark import SparkFiles
from pyspark.sql.functions import avg, col, desc
vine_table_HPC.groupBy("star_rating", "vine").count().sort(col("star_rating").desc()).show(truncate=False)

Average Number of helpful votes for both Vine Members & Non Members



In [ ]:
# Softwares
from pyspark.sql.types import IntegerType
from pyspark import SparkFiles
vine_table_softwares = vine_table_softwares.withColumn("helpful_votes",softwares_df["helpful_votes"].cast(IntegerType()))
vine_table_softwares.groupBy("vine").avg("helpful_votes").show(truncate=False)

In [ ]:
# Health Personal Care
from pyspark.sql.types import IntegerType
from pyspark import SparkFiles
vine_table_HPC = vine_table_watches.withColumn("helpful_votes",HPC_df["helpful_votes"].cast(IntegerType()))
vine_table_HPC.groupBy("vine").avg("helpful_votes").show(truncate=False)